In [ ]:
'''
Applying dwave-ocean-sdk (Quantum Annealer) to solve MCLP problem
Problem Statement:
Given N demand points and M service centers,find the least centers that can serve all demand points. 
Specifically, each center has a specific capacity, whereas each demand has a specific demand degree (weight).
'''

In [ ]:
#Number demand points and centers
num_demands = 17
num_centers = 5

#Weights of demand points, and capacity of centes (all centers have the same capabity： 25)
demand_weights = [3,4,5,6,7,4,2,3,4,5,6,3,5,4,3,5,1]
center_capacity = 25

#Locations of demand points and centers
demandCoordinates = [(88, 16),(25, 76),(69, 13),(73, 56),(80, 100),(22, 92),(32, 84),(73, 46),(29, 10),(92, 32),(44, 44),(55, 26),(71, 27),(51, 91),(89, 54),(43, 28),(40, 78)]
centerCoordinates = [(32, 60),(69, 33),(49, 40),(72, 81),(61, 65)]

#Euclidean distances between centers and demand points
import numpy as np
distances = np.zeros((17, 5))
for i in range(len(demandCoordinates)):
    for j in range(len(centerCoordinates)):
        distances[i][j] = np.sqrt(
            (demandCoordinates[i][0] - centerCoordinates[j][0]) ** 2 + 
            (demandCoordinates[i][1] - centerCoordinates[j][1]) ** 2)
        
print("Problem: Allocate demand points of total weight of {} into centers of capacity {}.".format(
      sum(demand_weights), center_capacity)) 

In [ ]:
#Define the Constrained Quadratic Model
from dimod import ConstrainedQuadraticModel
from dimod import Binary
cqm = ConstrainedQuadraticModel()

#dDefine the variable to indicate whether a center is selected
center_used = [Binary(f'center_used_{j}') for j in range(num_centers)]

#Define the variable to indicate whether a demand point_i in center_j
demand_in_center = [[Binary(f'demand_{i}_in_center_{j}') for j in range(num_centers)] for i in range(num_demands)]

#Calculate the total weright of allocated demand points
def total_demand_weights():
    res = 0
    for i in range(num_demands):
        for j in range(num_centers):
            res += demand_in_center[i][j] * demand_weights[i]
    return res

#Set the objective function: Minimize the selected centers while maximize the allocated demand points
cqm.set_objective(sum(center_used) - total_demand_weights())
#Note that if we hope that minimize the number of selected centers is more important, we can give it a coefficient such as 20.
#On the contrary, if we hope the total weights of arranged demands could be larger, we can give total_demand_weights() a coefficient.

In [ ]:
#Add constraints to the model

#First: each demand point can be allocated to only one center
for i in range(num_demands):
    one_center_per_demand = cqm.add_constraint(sum(demand_in_center[i]) - 1 <= 0, label=f'demand_allocated_{i}')
    
#Second: the capacity of center is limited to 25
for j in range(num_centers):
    demand_to_center_capacity = cqm.add_constraint(sum(demand_weights[i] * demand_in_center[i][j] for i in range(num_demands)) - center_used[j] * center_capacity <= 0,
                                                  label = f'capacity_center_{j}')

#Third: the distance between the center and demand points is constrainted to $delta (e.g. 50)
for j in range(num_centers):
    for i in range(num_demands):
        demand_to_center_distance = cqm.add_constraint(distances[i][j] * demand_in_center[i][j] - 50 <= 9, label = f'distance_of_demand_{i}_to_center_{j}')

In [ ]:
#Solve the model using LeapHybridQMSampler
from dwave.system import LeapHybridCQMSampler
sampler = LeapHybridCQMSampler()

sampleset = sampler.sample_cqm(cqm,
                               time_limit=180,
                               label="MCLP-Distance_Constraint")  
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)  
if len(feasible_sampleset):      
   best = feasible_sampleset.first
   print("{} feasible solutions of {}.".format(
      len(feasible_sampleset), len(sampleset)))

In [ ]:
#Find the selected centers
selected_centers = [key for key, val in best.sample.items() if 'center_used' in key and val]

def get_indices(name):
    return [int(digs) for digs in name.split('_') if digs.isdigit()]

for center in selected_centers:                        
     in_center = [key for key, val in best.sample.items() if "_in_center" in key and get_indices(key)[1] == get_indices(center)[0] and val]
     if (len(in_center) > 0):
        b = get_indices(in_center[0])[1]
        w = [demand_weights[get_indices(item)[0]] for item in in_center]
        print("Center {} has weights {} for a total of {}.".format(b, w, sum(w)))